# Filteren en sorteren 
We werken met de volgende gegevens: https://www.kaggle.com/api/v1/datasets/download/iamsouravbanerjee/world-population-dataset

In [ ]:
from pathlib import Path
from zipfile import ZipFile
import requests
POPULATION_ZIP = 'population.zip'
WORLD_POPULATION_CSV = 'world_population.csv'
population_path = Path(POPULATION_ZIP)
if not population_path.exists():
    URL= 'https://www.kaggle.com/api/v1/datasets/download/iamsouravbanerjee/world-population-dataset'
    data = requests.get(URL)
    with open(POPULATION_ZIP, mode='wb') as f:
        f.write(data.content)
else:
    print('Bestand moet niet gedownload worden')
world_population_path = Path(WORLD_POPULATION_CSV)
if not world_population_path.exists():
    with open(POPULATION_ZIP, 'rb') as f:
        ZipFile(f).extractall()
else:
    print('Bestand moet niet uitgepakt worden')

## Structuur
Ik lees de eerste regels weer met Python om een idee van de structuur te krijgen

In [ ]:
WORLD_POPULATION_CSV = 'world_population.csv'
with open(WORLD_POPULATION_CSV) as f:
    for _ in range(5):
        print(f.readline(), end='')


## Dataframe lezen
Een mogelijke kandidaat voor de index is de landcode van drie letters

In [ ]:
import pandas as pd
WORLD_POPULATION_CSV = 'world_population.csv'
df = pd.read_csv(WORLD_POPULATION_CSV, index_col='CCA3')
df.info()

## Aantal rijen
Hoeveel rijen zijn er?

In [ ]:
df.shape[0]

## Objectkolommen omzetten naar string
Met .select_dtypes kunnen we alle kolommen van een bepaald type selecteren. Hier selecteren we de objectkolommen ('O')

In [ ]:
objectkolommen = df.select_dtypes(include='O').columns
stringtypes = {kolom:pd.StringDtype() for kolom in objectkolommen}
df = df.astype(stringtypes)
df.info()

## Enkele rijen afdrukken: .head() en .tail()
Met head() kunnen we de eerste rijen afdrukken. Wanneer we ook de tail() afdrukken, zien we dat het dataframe waarschijnlijk gesorteerd is op Country/Territory

In [ ]:
display(df.head())
display(df.tail())

## Een idee krijgen van de data
De shape kennen we al. Een volgende logische stap is de .describe()-functie. De Stringkolommen zijn hier niet direct interessant.

In [ ]:
df.describe()

## Welke landen hadden meer dan 1.000.000.000 inwoners in 2022?
Dat is een filtering die we al gezien hebben

In [ ]:
df[df['2022 Population'] > 1_000_000_000]

## Welk land heeft de grootste growth rate
We kunnen dat oplossen met een filter

In [ ]:
max_growth_rate = df['Growth Rate'].max()
df[df['Growth Rate'] == max_growth_rate]


## Een alternatief
We zijn hier twee floats aan het vergelijken. Dat zal wel werken omdat er geen berekeningen gebeuren. We vergelijken een getal met zichzelf. Maar een alternatief is sorteren.

In [ ]:
df.sort_values('Growth Rate', ascending=False)[:5]

## Op basis van wat is de Rank gebaseerd?
Heeft de rank te maken met de 2022 population? Dat kunnen we zien door oplopend te sorteren op Rank.

In [ ]:
df.sort_values('Rank')

## We kunnen ook de index wisselen en sorteren op index
Sorteren op een index is efficiënter. Dus we kunnen ook de index wijzigen.

In [ ]:
df = df.reset_index()
df = df.set_index('Rank')
df.sort_index()

## Sorteren op basis van een 'key'
De sorted()-functie van Python kan sorteren op een 'key': een functie die de waarde teruggeeft waarop gesorteerd moet worden. Stel dat we willen sorteren op de lengte van Country/Territory

In [ ]:
df.sort_values(by='Country/Territory',key=lambda cr: cr.str.len())

## Sorteren op meerdere velden
We kunnen eerst sorteren op Continent en vervolgens op '2022 Population'. Het ascending-argument kan dan een list van booleans worden: aflopen sorteren op Continent, en oplopend op 2022 Population

In [ ]:
df.sort_values(by=['Continent', '2022 Population'], ascending=[False, True])

## En hoe zit het weer met NULL_waarden
Deze dataset bevat geen NULL waarden, maar daar kunnen we wel voor zorgen natuurlijk. (denk aan de CopyOnWrite warning)
We geven alle landen waarbij World Population Percentage gelijk is aan 0 de waarde pd.NA voor die kolom. Dat is voor deze kolom geen probleem omdat het een float-kolom is.

In [ ]:
df.loc[df['World Population Percentage']==0, 'World Population Percentage'] = pd.NA
df.info()

## filteren op lege waarden
We kunnen hiervoor isnull() of isna() gebruiken.

In [ ]:
df[df['World Population Percentage'].isnull()]